In [1]:
#Kmer generator
def getKmers(sequence, size, step): 
    output_seq=[]
    for x in range(0, len(sequence) - size, step):
        output_seq.append(sequence[x:x+size])
    return(output_seq)

In [64]:
#Training with three models
import matplotlib.pyplot as mp
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn import datasets
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import numpy as np 
from scipy import stats

KMER_RANGE=range(5,9,3)
REPLICA=5
for k in range(3,4):
    import pandas as pd
    df_input=pd.read_csv("/Users/moradigd/Documents/Plasmid/input_file.csv")
    print("Prediction for pasmid "+ df_input.columns[k])
    df_input=df_input.iloc[:,[0,k]]
    df_input=df_input.dropna()
    df_input=df_input.reset_index()
    
    for g in KMER_RANGE:
        result_f=[]
        print("start counting kmers of size "+ str(g))
        unique_sequences_set=[]
        for i in range(df_input.shape[0]):
            unique_sequences_set=unique_sequences_set+getKmers(df_input['seq'][i], g, 1)
        unique_sequences=list(set(unique_sequences_set))
        [df_input['seq'][1].count(i) for i in unique_sequences]
        print("done")

        print("start creating kmer profile")
        unique_sequences_freq=[]
        for j in range(df_input.shape[0]):
            unique_sequences_freq.append([df_input['seq'][j].count(i) for i in unique_sequences])
        unique_sequences_freq   
        unique_sequences_freq=pd.DataFrame(unique_sequences_freq)
        unique_sequences_freq
        print("done")

        unique_sequences_freq['label']=df_input.iloc[0:df_input.shape[0],2]
        scaler = MinMaxScaler()
        unique_sequences_freq = pd.DataFrame(scaler.fit_transform(unique_sequences_freq), columns=unique_sequences_freq.columns)

        data=unique_sequences_freq.values
        X, y = data[:, :-1], data[:, -1]
        print("start prediction")
        for m in range(REPLICA):
            print("Prediction replicate "+ str(m))
            
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
            #gradient boosting regressor
            print("start prediction with gradient boosting regressor")
            parameters = {
            "max_depth":[1,3,5],
            "subsample":[ 0.8, 1.0],
            "n_estimators":[5,10,30,100]
            }

            clf = GridSearchCV(GradientBoostingRegressor(), parameters, cv=3, n_jobs=-1)
            clf.fit(X_train, y_train)
            result = clf.predict(X_test)
            
            df_imporatnce=pd.DataFrame({"tag":unique_sequences, "importance":clf.best_estimator_.feature_importances_})
            df_imporatnce=df_imporatnce.loc[df_imporatnce['importance'] > 0]
            df_imporatnce.to_csv("/Users/moradigd/Documents/Plasmid/Kmer_importance_"+str(g)+"_"+str(m)+'_'+str(df_input.columns[2]))

            rho, pval = stats.spearmanr(result, y_test)
            result_f.append([rho,pval, g, df_input.columns[2], 'gb',m])
            
            #random forests
            print("start prediction with random forests regressor")
            parameters = {
            "min_samples_split":[1,3,5],
            "subsample":[ 0.8, 1.0],
            "n_estimators":[5,10,100]
            }

            clf = GridSearchCV(RandomForestRegressor(), parameters, cv=3, n_jobs=-1)
            clf.fit(X_train, y_train)
            result = clf.predict(X_test)
            
            df_imporatnce=pd.DataFrame({"tag":Onehot_agg.columns[0:len(Onehot_agg.columns)-1].values, "importance":clf.best_estimator_.feature_importances_})
            df_imporatnce=df_imporatnce.loc[df_imporatnce['importance'] > 0]
            df_imporatnce.to_csv("/Users/moradigd/Documents/Plasmid/PB_baps_importance_"+str(g)+"_"+str(m)+'_'+str(df_input.columns[0]))
    
            rho, pval = stats.spearmanr(result, y_test)
            result_f.append([rho,pval, g, df_input.columns[2], 'rf',m])

            #Lasso 
            print("start prediction with lasso regressor")
            lasso = Lasso(random_state=0, max_iter=10000)
            alphas = np.logspace(-4, -0.5, 10)
    
            tuned_parameters = [{'alpha': alphas}]
            clf = GridSearchCV(lasso, tuned_parameters, cv=3)
            clf.fit(X_train, y_train)
            result = clf.predict(X_test)

            rho, pval = stats.spearmanr(result, y_test)
            result_f.append([rho,pval, g, df_input.columns[2], 'lasso',m])
            
        pd.DataFrame(result_f).to_csv("/Users/moradigd/Documents/Plasmid/New_results_screening_agg_scale_"+str(g)+'_'+str(df_input.columns[2])+".csv")


Index(['seq', 'PKJK', 'RP', 'PB'], dtype='object')
start
5
done
start
done
0.44289074812132057
{'max_depth': 5, 'n_estimators': 5, 'subsample': 0.8}
       tag  importance
11   ACCAC    0.015224
21   ACCTG    0.181398
35   AGGGC    0.015353
67   TGGTG    0.010615
83   GGGTT    0.008673
116  CTGAG    0.009374
149  GTTGG    0.029025
151  ATTGC    0.039520
185  GCGTT    0.000788
212  CGACA    0.018507
236  TATGC    0.026292
251  ATGGA    0.007189
269  GGGCA    0.004117
346  TAGTT    0.013643
370  GCTTT    0.036445
377  GGCAG    0.023400
443  CTTAC    0.000827
483  TTATA    0.000846
520  TTGGA    0.013661
527  GAAGG    0.018954
553  ACGAA    0.008961
562  CCTAA    0.008364
564  CGGTA    0.000545
571  GTCTT    0.004916
572  GTTAC    0.073461
593  CAAAC    0.007718
623  TGGAT    0.017128
668  TTTAG    0.028953
714  TTAGT    0.047789
769  GTTTG    0.031714
789  TTTTG    0.049073
796  GCTCG    0.015727
856  TTGCT    0.060714
864  TTACC    0.077937
885  CGCCG    0.022716
915  CAGTA    0.004306


WorkerInterrupt: 

In [ ]:
#Subset simulation
import matplotlib.pyplot as mp
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn import datasets
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV0-
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import numpy as np 
from scipy import stats
import pandas as pd

SUBSIZE=np.arange(0.1,1,0.1)
KMER_RANGE=range(5,9,3)
REPLICA=5
for k in range(1,4):
    df_input=pd.read_csv("/Users/moradigd/Documents/Plasmid/input_file")
    print("Prediction for pasmid "+ df_input.columns[k])
    df_input=df_input.iloc[:,[0,k]]
    df_input.shape
    df_input=df_input.dropna()
    df_input=df_input.reset_index()
    
    for g in KMER_RANGE:
        result_f=[]
        print("start counting kmers of size "+ str(g))
        unique_sequences_set=[]
        for i in range(df_input.shape[0]):
            unique_sequences_set=unique_sequences_set+getKmers(df_input['seq'][i], g, 1)
        unique_sequences=list(set(unique_sequences_set))
        [df_input['seq'][1].count(i) for i in unique_sequences]
        print("done")
        
        print("start creating kmer profile")
        unique_sequences_freq=[]
        for j in range(df_input.shape[0]):
            unique_sequences_freq.append([df_input['seq'][j].count(i) for i in unique_sequences])
        unique_sequences_freq   
        unique_sequences_freq=pd.DataFrame(unique_sequences_freq)
        unique_sequences_freq
        print("done")


        unique_sequences_freq[['label']]=df_input.iloc[0:df_input.shape[0],2]        
        scaler = MinMaxScaler()
        unique_sequences_freq = pd.DataFrame(scaler.fit_transform(unique_sequences_freq), columns=unique_sequences_freq.columns)
        data=unique_sequences_freq.values
        
        print("start prediction with random forests")
        X, y = data[:, :-1], data[:, -1]
        for m in range(REPLICA):
            print("Prediction replicate "+str(m))
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
            for k in range(len(SUBSIZE)):
                print("Prediction sub-sample of "+str(SUBSIZE[k]))
                selected_ids=np.random.choice(X_train.shape[0], int(SUBSIZE[k]*X_train.shape[0]) , replace=False)
                X_train_tmp=X_train[selected_ids]
                y_train_tmp=y_train[selected_ids]

                parameters = {
                "min_samples_split":[1,3,5],
                "subsample":[ 0.8, 1.0],
                "n_estimators":[5,10,100]
                }

                clf = GridSearchCV(RandomForestRegressor(), parameters, cv=3, n_jobs=-1)
                clf.fit(X_train_tmp, y_train_tmp)
                result = clf.predict(X_test)

                rho, pval = stats.spearmanr(result, y_test)
                result_f.append([rho,pval, g, df_input.columns[2], 'rf',m,subsize[k], "test"])
                
                result = clf.predict(X_train_tmp)
                rho, pval = stats.spearmanr(result, y_train_tmp)
                result_f.append([rho,pval, g, df_input.columns[2], 'rf',m,subsize[k], "train"])

        pd.DataFrame(result_f).to_csv("/Users/moradigd/Documents/Plasmid/Subset_results_"+str(g)+'_'+str(df_input.columns[2])+"_"+str(SUBSIZE[k])+".csv")
        